In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [ ]:
names_df = pd.read_csv('./Seoul.csv')
names_df

,0
0,강남구 개포동 레몬그라스 타이
1,강남구 개포동 장어미식가 개포2호점
2,강남구 개포동 교촌치킨 개포2호점
3,강남구 개포동 도헤(Dorer)
4,강남구 개포동 돈바치
...,...
386,강남구 개포동 비비큐BBQ 개포대청역점
387,강남구 개포동 북경
388,강남구 개포동 혼(Hone)
389,강남구 개포동 윤이네


In [ ]:
# 창이 나오지 않게 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver.exe' # 크롬드라이버 경로 설정

source_url = "https://map.kakao.com/" # 리뷰주소 크롤링을 해 올 주소 설정

page_urls = [] #리뷰주소 크롤링 결과를 담을 리스트

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in tqdm_notebook(names_df['0']):
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창 찾기
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색 버튼 누르기
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source # 페이지 소스 설정

        soup = BeautifulSoup(html, "html.parser") # 페이지 소스 파싱
        moreview = soup.find(name="a", attrs={"class":"moreview"}) # 상세 보기 찾기
    
        page_url = moreview.get("href") # 상세보기에 링크된 리뷰주소 가져오기
        print(page_url)
        page_urls.append(page_url) 
    except:
        pass

driver.close()

  0%|          | 0/391 [00:00<?, ?it/s]

https://place.map.kakao.com/207990934
https://place.map.kakao.com/1441083691
https://place.map.kakao.com/1605389572
https://place.map.kakao.com/908713570
https://place.map.kakao.com/16738793
https://place.map.kakao.com/21413534
https://place.map.kakao.com/1824464463
https://place.map.kakao.com/15558781
https://place.map.kakao.com/21753236
https://place.map.kakao.com/8607176
https://place.map.kakao.com/274655501
https://place.map.kakao.com/7902292
https://place.map.kakao.com/16296316
https://place.map.kakao.com/1367052337
https://place.map.kakao.com/21129020
https://place.map.kakao.com/10028218
https://place.map.kakao.com/16036326
https://place.map.kakao.com/16875544
https://place.map.kakao.com/7882274
https://place.map.kakao.com/20745142
https://place.map.kakao.com/626539900
https://place.map.kakao.com/2056915027
https://place.map.kakao.com/19009665
https://place.map.kakao.com/27297148
https://place.map.kakao.com/1696150513
https://place.map.kakao.com/19010584
https://place.map.kakao.c

In [ ]:
# 이미 리뷰주소가 있을 경우 사용
'''page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver' '''

In [ ]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2'))) # 가게 이름이 보일 때까지
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"}) # 가게 이름 찾기
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"}) # 후기 작성란 찾기
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"}) # 별점 찾기
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"}) # 리뷰 찾기
    
        # 첫번째 페이지 별점 및 리뷰 가져오기
        for rate, review in zip(rates, reviews):
            if review != ' ':
                row = [names.text, rate.text[0], review.find(name="span").text] # 가게이름, 별점, 리뷰 순으로 가져와 리스트 만들기
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)
        
        # 두번째 페이지부터 다섯번째 페이지까지 별점 및 리뷰 가져오기
        for button_num in range(2, 6): 
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']") # 페이지 넘기기
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                if review != ' ':
                    row = [names.text, rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=review_df.columns)
                    review_df = review_df.append(series, ignore_index=True)
    except:
        pass # 리뷰가 없거나 리뷰를 공개하지 않은 곳들도 있어서 패스 설정
    
driver.close()

  0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
# 리뷰 없이 별점만 남긴 리뷰 제거
filter = review_df["review"] != "" 
review_data = review_df[filter]
review_data

,name,score,review
0,돈가네 개포본점,5,개포동에서 제일오래된집 믿고먹는 돼지찌개 사장님과 직원분들이 친절해서 더좋음
1,돈가네 개포본점,5,김치찌개 푸짐하고 맛있어요. 직원분들도 너무 친절하셔서 기분좋게 먹고 왔네요.
2,돈가네 개포본점,5,항상 갑니다~ 김치찌개 정말 맛있어요!
4,어머니밥상,1,살다살다 이렇게 더러운덴 처음봄
5,원조소금구이 제2호점,5,굿굿 고기좋네요 기본으로 나오는 된장찌개가 압권
...,...,...,...
428,북경,5,빠른배달 역쉬 뿌른면은 맛이 없죠. 맛있어요
429,북경,1,솔직히 1점 주는것도 아깝습니다. 11시 43분에 마파두부밥 하나 주문했는데 12시...
431,북경,5,금요일메뉴 마파두부밥 7천
432,북경,1,"12시 20분에 주문, 1시에 확인하니 출발했다고 하더라구요? 20분이 지나도 안와..."


In [ ]:
review_data.to_csv("./review_data.csv", index=False, encoding='utf-8-sig')